### 1. Implementing Agents with Langchain
Install commonly used libraries.

In [ ]:
!pip install langchain openai wikipedia langchain-community "httpx==0.27.2" # Use '%pip install' or '!pip install'

In [ ]:
import os
from langchain.chat_models import ChatOpenAI

os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY') # Enter an API starting with 'sk-'

llm = ChatOpenAI(model="gpt-4o-2024-11-20")

### (1) Zero-shot ReAct
The agent to be used is Zero-shot ReAct.

To do this, you need to specify which tool to use.

We chose Wikipedia for search purposes and llm-math for mathematical questions.


In [ ]:
from langchain.agents import initialize_agent, load_tools, AgentType

tools = load_tools(["wikipedia" ,"llm-math"], llm=llm)

In [ ]:
agent = initialize_agent(tools,
                         llm,
                         agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
                         handle_parsing_errors=True,
                         verbose = True) # Initialize conversational agent

# Send questions to the agent and generate responses.
agent.invoke("When is the best season for exercising?")

### (2) Conversational ReAct

Next, let's apply the Conversational ReAct type.

To use this agent, enter agent=‘conversational-react-description’.

In the following example, we will ask Ed Sheeran's age because we used a tool called llm-math designed to solve math problems.

In [ ]:
from langchain.memory import ConversationBufferMemory

# Specify tools for agents to use Tools for solving math-related problems
tools = load_tools(["llm-math"], llm=llm)
memory = ConversationBufferMemory(memory_key="chat_history")

# Specify keys for saving conversation records
conversational_agent = initialize_agent(agent='conversational-react-description',
                                        tools=tools,
                                        llm=llm,
                                        verbose=True, # Output agent internal operations for debugging purposes,
                                        max_iterations=3, # Maximum number of times an agent can use a tool
                                        memory=memory)
conversational_agent.invoke("Who is Ed Sheeran, and how old is he in 2025?")

### (3) Self-ask with search

The following is Self-ask with search agent.
First, import the necessary libraries.

As explained earlier, TavilyAnswer is a library used for searching.

In [ ]:
from langchain import hub
from langchain.agents import AgentExecutor, create_self_ask_with_search_agent
from langchain_community.tools.tavily_search import TavilyAnswer

In [ ]:
import os
os.environ["TAVILY_API_KEY"] = "tvly-" # Enter the API key starting with tvly

In this example, we use the AgentExecutor function to use the agent.

Let's ask the question, “Describe AI Agent in three sentences.”

In [ ]:
# Set the tool name so that the agent can use it when referring to this tool later.
tools = [TavilyAnswer(max_results=5, name="Intermediate Answer")]

# Get the self-ask-with-search prompt from the Langchain Prompt Hub.
prompt = hub.pull("hwchase17/self-ask-with-search")

# Create an agent that implements the self-ask with search function.
agent = create_self_ask_with_search_agent(llm, tools, prompt)

agent_executor = AgentExecutor(
	agent=agent,
	tools=tools,
	handel_parsing_errors=True
) # Using AgentExecutor to use agents

agent_executor.invoke({"input": "Describe AI Agent in three sentences."})


### (4) ReAct docstore
Finally, there is ReAct docstore.

This time, we will learn how to search and view documents on Wikipedia using an agent.

First, explore the Wikipedia document repository and set up tools that allow you to search and look up documents.

In [ ]:
# Using Wikipedia as a document repository
from langchain.docstore.wikipedia import Wikipedia
from langchain.agents import Tool

In [ ]:
# To explore the database where the documents are stored (Wikipedia)
from langchain.agents.react.base import DocstoreExplorer
docstore = DocstoreExplorer(Wikipedia())
tools = [
    Tool(
        name="Search",
        func=docstore.search, # Keyword-based search
        description="Search for terms in docstore.",
    ),
    Tool(
        name = "Lookup",
        func=docstore.lookup, # Search using uniquely identifiable keys
        description="Search for definitions of terms in docstore.",
    )
]   # Define tools available to agents

Initialize the langchain.

In [ ]:
react = initialize_agent(
    tools = tools,
    llm = llm,
    agent = AgentType.REACT_DOCSTORE, # Specify agent type
    handle_parsing_errors = True, # Handle syntax errors that occur during operation
    max_iterations=5, # Specify the maximum number of times to repeat the operation
    max_execution_time=5, # Set the maximum operation time in seconds
    verbose=True, # Output the internal operations of the agent for debugging purposes
) # Initialize the Langchain agent

Use an agent to search for and output answers to user questions.

In [ ]:
def query_data(query):
    try:
        response = react.invoke(query) # Send query to agent
        print("-----------", response) # Output results (for debugging)
        return response # Return search results
    except Exception as e:
        print(f"Error: {e}") # Output error message
        raise # Repeat error (for additional debugging)

query = "What is the best way to exercise in the hot summer?"
response = query_data(query)
print(response['output']) # Output agent's final answer

In [ ]:
!rm -rf Auto-GPT